In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 4- 4.1 Segmentação de Politica Novos

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({
                             'payloadHomol.intermediarias.clienteFoPag':[0],
                             'solicitante.rendaLiquidaPicpay':[10000],
                             'payloadHomol.intermediarias.flagClienteExistente':[0]
                            })

In [0]:
#Criação das massas com casos forçados
massa_SegmentacaoPoliticaMarAberto = sherlock.Gera_Massa(base_entrada)
massa_SegmentacaoPoliticaMarAberto.gera_cenario('payloadHomol.intermediarias.flagClienteExistente',[0,1])
massa_SegmentacaoPoliticaMarAberto.gera_cenario('payloadHomol.intermediarias.clienteFoPag',[0,1])
massa_SegmentacaoPoliticaMarAberto.gera_cenario('solicitante.rendaLiquidaPicpay',[-99999,0,3442.36,3442.37,3442.38,11124.15,11124.16,11124.17,100000])

massa_SegmentacaoPoliticaMarAberto.get_massa_final()

massa_SegmentacaoPoliticaMarAberto.set_bom_arqv()
massa_SegmentacaoPoliticaMarAberto.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_SegmentacaoPoliticaMarAberto.get_massa_json(),rp.SegmentacaoPoliticaMarAberto)

##### Valida se todas condições estão sendo abordadas

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['segmentacaoPolitica'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_SegmentacaoPoliticaMarAberto.rename_cabecalho()

In [0]:
massa_SegmentacaoPoliticaMarAberto.get_massa_dm()

In [0]:
massa_SegmentacaoPoliticaMarAberto.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacaoPolitica","segmentacaoPolitica")
massa_SegmentacaoPoliticaMarAberto.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_SegmentacaoPoliticaMarAberto.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaSegmentacaoPoliticaMarAberto.csv',index=False)